# Assignment 1


In [2]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

/home/jorge/.local/share/virtualenvs/RecommenderSystems-c5N1t04d/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [36]:
# create inverted index to help recover the words from indexes
inverted_index = {}
for word, i in tokenizer.word_index.items():
    inverted_index[i] = word
    
print(inverted_index)

{1: 'the', 2: 'and', 3: 'to', 4: 'she', 5: 'a', 6: 'i', 7: 'it', 8: 'of', 9: 'was', 10: 'in', 11: 'alice', 12: 'you', 13: 'that', 14: 'her', 15: 'as', 16: 'said', 17: 'had', 18: 'for', 19: 'but', 20: 'be', 21: 'on', 22: 'all', 23: 'with', 24: 'little', 25: 'mouse', 26: 'down', 27: 'very', 28: 'this', 29: 'not', 30: 'so', 31: 'out', 32: 'if', 33: 'is', 34: 'at', 35: 't', 36: 's', 37: 'll', 38: 'how', 39: 'they', 40: 'about', 41: 'herself', 42: 'me', 43: 'up', 44: 'what', 45: 'way', 46: 'when', 47: 'like', 48: 'one', 49: 'do', 50: 'no', 51: 'oh', 52: 'went', 53: 'thought', 54: 'again', 55: 'there', 56: 'see', 57: 'or', 58: 'could', 59: 'would', 60: 'think', 61: 'them', 62: 'know', 63: 'rabbit', 64: 'dear', 65: 'were', 66: 'time', 67: 'get', 68: 'here', 69: 'must', 70: 'my', 71: 'by', 72: 'into', 73: 'found', 74: 'such', 75: 'began', 76: 'soon', 77: 'm', 78: 'quite', 79: 'then', 80: 'off', 81: 'now', 82: 'go', 83: 'say', 84: 'have', 85: 'which', 86: 'come', 87: 'dinah', 88: 'your', 89: 't

In [24]:
################################
# helper snippet
################################

m = np.matrix([[1, 2], [3, 4]])
print(m)
(rows, columns) = m.shape
for rowIdx in range(rows):
    s = m[rowIdx]
    total = s.sum()
    print(s.sum())
    print(np.divide(s, total))
    print(s.dtype)
    print(s.flatten())
    
    #for value in m[rowIdx, :]:
    #    print(value)
    #print(m[rowIdx, :])
    



[[1 2]
 [3 4]]
3
[[0.33333333 0.66666667]]
int64
[[1 2]]
7
[[0.42857143 0.57142857]]
int64
[[3 4]]


In [35]:
############################
# helper snippet 
############################
nan_indexes = [797, 1090, 1093, 1094, 1095, 1096, 1097]
for sequence in corpus:
    for nan_index in nan_indexes:
        if nan_index in sequence:
            print(sequence)
            break
    
print(tokenizer.word_index)

[797]
[1090]
[1093]
[1094]
[1095]
[1096]
[1097]
{'duck': 374, 'begged': 1081, 'nearly': 871, 'advise': 752, 'have': 84, 'fear': 402, 'heap': 660, 'beasts': 709, 'drink': 338, 'met': 1087, 'choked': 1079, 'chorus': 565, 'pairs': 1182, 'simply': 1071, 'w': 1170, 'way': 45, 'longed': 688, 'leap': 921, 'politely': 999, 'then': 79, 'odd': 794, 'neatly': 854, 'key': 148, 'gave': 474, 'red': 715, 'shrill': 926, 'swim': 1156, 'growled': 1109, 'english': 228, 'hour': 1041, 'continued': 1020, 'nicely': 934, 'side': 441, 'these': 881, 'country': 643, 'shrinking': 517, 'claws': 856, 'jar': 401, 'stairs': 627, 'bed': 1139, 'hippopotamus': 906, 'i': 6, 'bathing': 891, 'hanging': 670, 'despair': 1054, 'conversation': 944, 'times': 359, 'remember': 219, 'sharply': 471, 'miles': 407, 'fur': 546, 'find': 150, 'australia': 645, 'playing': 761, 'getting': 138, 'dark': 305, 'harm': 907, 'various': 1140, 'sight': 328, 'nothing': 112, 'trying': 187, 'four': 214, 'boots': 789, 'child': 477, 'speech': 567, 'th

In [40]:
#create co-occurrence matrix
#print(corpus[0])
coMatrix = np.zeros((V, V))
for sequence in corpus:
    for idx, termId in enumerate(sequence):
        sl = len(sequence)
        ## select left window
        leftw = sequence[max(idx - window_size_corpus, 0): idx]
        ## select right window
        rightw = sequence[idx + 1: min(idx + window_size_corpus + 1, sl)]
        # update co-occurrence matrix
        neighboors = leftw + rightw
        for neighbor in neighboors:
            coMatrix[termId, neighbor] += 1
        

# matrix normalization
(rows, columns) = coMatrix.shape
for rowIdx in range(rows):
    # ignore first row with 0 entries everywhere
    if (rowIdx > 0):
        row = coMatrix[rowIdx]
        total = row.sum()
        # avoid division by zero in words that have no neighboors
        if total > 0:
            coMatrix[rowIdx] = np.divide(row, total)


[242, 6, 26, 1, 63, 243]


In [28]:
print(coMatrix)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.02363823 0.0364851  ... 0.00051387 0.         0.        ]
 [0.         0.04752343 0.01204819 ... 0.00066934 0.00066934 0.00066934]
 ...
 [0.         0.16666667 0.16666667 ... 0.         0.         0.        ]
 [0.         0.         0.125      ... 0.         0.         0.        ]
 [0.         0.         0.25       ... 0.         0.         0.        ]]


In [48]:
#find cosine similarity to Alice, Dinah and Rabbit

#find the word vectors for Alice, Dinah, and Rabbit
aliceIdx = tokenizer.word_index['Alice'.lower()]
dinahIdx = tokenizer.word_index['Dinah'.lower()]
rabbitIdx = tokenizer.word_index['Rabbit'.lower()]

aliceVector = coMatrix[aliceIdx].reshape(1, -1)
dinahVector = coMatrix[dinahIdx].reshape(1, -1)
rabbitVector = coMatrix[rabbitIdx].reshape(1, -1)
print(aliceVector)
print(dinahVector)
print(rabbitVector)

cosAD = cosine_similarity(aliceVector, dinahVector)
print("Cosine similarity between Alice and Dinah: {}".format(cosAD))
cosAR = cosine_similarity(aliceVector, rabbitVector)
print("Cosine similarity between Alice and Rabbit: {}".format(cosAR))
cosDR = cosine_similarity(dinahVector, rabbitVector)
print("Cosine similarity between Dinah and Rabbit: {}".format(cosDR))


[[0.         0.02646503 0.03402647 ... 0.         0.         0.        ]]
[[0.         0.03448276 0.03448276 ... 0.         0.         0.        ]]
[[0.         0.13559322 0.00847458 ... 0.         0.         0.        ]]
Cosine similarity between Alice and Dinah: [[0.39360011]]
Cosine similarity between Alice and Rabbit: [[0.47890931]]
Cosine similarity between Dinah and Rabbit: [[0.29862324]]


In [49]:
#find the closest words to Alice
# create an array containing the cosine similarity values for alice and the rest
nbrs = nn(n_neighbors=6, algorithm='brute', metric='cosine').fit(coMatrix)
distances, indices = nbrs.kneighbors(aliceVector)
print(distances)
print(indices)


[[0.         0.18693993 0.22482059 0.25996004 0.26423806 0.2653115 ]]
[[11  4  7  1  5 41]]


In [ ]:
# validate the computed distances and the indices

From the results we observe the six closest words to 'Alice'.

Word with index 11 corresponds to 'Alice' itself. 

The words for the remaining indexes are:
* 4: she
* 7: it
* 1: the
* 5: a
* 41: herself

Discussion of the drawbacks:

In [ ]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = []
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [ ]:
#reopen your file as follows

co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [3]:
#prepare data for cbow


In [242]:
#create CBOW model


In [243]:
#define loss function


In [1]:
#train model


In [4]:
#prepare data for Skipgram
def generate_data_skipgram(corpus, window_size, V):
    maxlen = window_size*2
    all_in = []
    all_out = []
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            p = index - window_size
            n = index + window_size + 1
            
            in_words = []
            labels = []
            for i in range(p, n):
                if i != index and 0 <= i < L:
                    # repeat the same word several times
                    in_words.append([word])
                    # add the context words
                    labels.append(words[i])
            if in_words != []:
                #print(in_words)
                all_in.append(np.array(in_words,dtype=np.int32))
                all_out.append(np_utils.to_categorical(labels, V))
                #print(all_in)
                #print(all_in[0].shape)
                #print(all_out)
                #print(all_out[0].shape)
                #break
    return (all_in,all_out)

In [5]:
#save the preprocessed data of Skipgram
def save_skipgram_data(filename, x, y):
    f = open(filename ,'w')
    for input,outcome  in zip(x,y):
        input = np.concatenate(input)
        f.write(" ".join(map(str, list(input))))
        f.write(",")
        outcome = np.concatenate(outcome)
        f.write(" ".join(map(str,list(outcome))))
        f.write("\n")
    f.close()

In [12]:
#load the preprocessed Skipgram data
def generate_data_skipgram_from_file(filename):
    f = open(filename ,'r')
    for row in f:
        inputs,outputs = row.split(",")
        inputs = np.fromstring(inputs, dtype=int, sep=' ')
        inputs = np.asarray(np.split(inputs, len(inputs)))
        outputs = np.fromstring(outputs, dtype=float, sep=' ')
        outputs = np.asarray(np.split(outputs, len(inputs)))
        yield (inputs,outputs)

In [8]:
SKIPGRAM_2WORDS = "data_skipgram_2words.txt"
x,y = generate_data_skipgram(corpus,2,V)
save_skipgram_data(SKIPGRAM_2WORDS, x, y)

SKIPGRAM_4WORDS = "data_skipgram_4words.txt"
x,y = generate_data_skipgram(corpus,4,V)
save_skipgram_data(SKIPGRAM_4WORDS, x, y)


In [9]:
#create Skipgram models
skipgram_2words_model = Sequential()
skipgram_2words_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_2words_model.add(Reshape((dim, )))
skipgram_2words_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_4words_model = Sequential()
skipgram_4words_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_4words_model.add(Reshape((dim, )))
skipgram_4words_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

In [10]:
#define loss function for Skipgrams
skipgram_2words_model.compile(loss='categorical_crossentropy', optimizer='adam')
#define loss function for Skipgram
skipgram_4words_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [25]:
#train Skipgram model
EPOCHS = 15

skipgram_2words_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_2WORDS):
        loss += skipgram_2words_model.train_on_batch(x, y)
    skipgram_2words_losses.append((ite, loss))
    print("skipgram 2words losses: {} -> {}".format(ite, loss))


skipgram_4words_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_4WORDS):
        loss += skipgram_4words_model.train_on_batch(x, y)
    skipgram_4words_losses.append((ite, loss))
    print("skipgram 4words losses: {} -> {}".format(ite, loss))


skipgram 2words losses: 0 -> 27182.093020916167
skipgram 2words losses: 1 -> 27199.516684770726
skipgram 2words losses: 2 -> 27204.492714703203
skipgram 2words losses: 3 -> 27218.393353700736
skipgram 2words losses: 4 -> 27224.77045786388
skipgram 2words losses: 5 -> 27239.30565458543
skipgram 2words losses: 6 -> 27243.966240525297
skipgram 2words losses: 7 -> 27260.401532769247
skipgram 2words losses: 8 -> 27263.237661421335
skipgram 2words losses: 9 -> 27279.125965714484
skipgram 2words losses: 10 -> 27284.51003623011
skipgram 2words losses: 11 -> 27297.318689405944
skipgram 2words losses: 12 -> 27301.81248903276
skipgram 2words losses: 13 -> 27316.966321110744
skipgram 2words losses: 14 -> 27319.665404140964
skipgram 4words losses: 0 -> 29733.113516869314
skipgram 4words losses: 1 -> 29745.975221336837
skipgram 4words losses: 2 -> 29753.071226538235
skipgram 4words losses: 3 -> 29765.893893063563
skipgram 4words losses: 4 -> 29774.47852456633
skipgram 4words losses: 5 -> 29787.55409

In [26]:
def save_computed_vectors(filename, vectors):
    f = open(filename ,'w')
    f.write(" ".join([str(V-1),str(dim)]))
    f.write("\n")

    for word, i in tokenizer.word_index.items():
        f.write(word)
        f.write(" ")
        f.write(" ".join(map(str, list(vectors[i,:]))))
        f.write("\n")
    f.close()

In [27]:
#Save computed vectors
VECTORS_SKIPGRAM_2WORDS_FILENAME = 'vectors_skipgram_2words.txt'
skipgram_2words_model_vectors = skipgram_2words_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_2WORDS_FILENAME, skipgram_2words_model_vectors)

VECTORS_SKIPGRAM_4WORDS_FILENAME = 'vectors_skipgram_4words.txt'
skipgram_4words_model_vectors = skipgram_4words_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_4WORDS_FILENAME, skipgram_4words_model_vectors)

In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [28]:
#create Skipgram models with additional dense layer

skipgram_2words_modified_model = Sequential()
skipgram_2words_modified_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_2words_modified_model.add(Reshape((dim, )))
skipgram_2words_modified_model.add(Dense(input_dim=dim, units=dim, kernel_initializer='he_uniform', activation='relu'))
skipgram_2words_modified_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))

skipgram_4words_modified_model = Sequential()
skipgram_4words_modified_model.add(Embedding(input_dim=V, output_dim=dim, embeddings_initializer='glorot_uniform', input_length=1))
skipgram_4words_modified_model.add(Reshape((dim, )))
skipgram_4words_modified_model.add(Dense(input_dim=dim, units=dim, kernel_initializer='he_uniform', activation='relu'))
skipgram_4words_modified_model.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))


In [29]:
#define loss function for Skipgram + dense
skipgram_2words_modified_model.compile(loss='categorical_crossentropy', optimizer='adam')
skipgram_4words_modified_model.compile(loss='categorical_crossentropy', optimizer='adam')

In [30]:
#train model for Skipgram + dense
EPOCHS = 15

skipgram_2words_modified_model_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_2WORDS):
        loss += skipgram_2words_modified_model.train_on_batch(x, y)
    skipgram_2words_modified_model_losses.append((ite, loss))
    print("skipgram 2words modified losses: {} -> {}".format(ite, loss))


skipgram_4words_modified_model_losses = []
for ite in range(EPOCHS):
    loss = 0.
    for x, y in generate_data_skipgram_from_file(SKIPGRAM_4WORDS):
        loss += skipgram_4words_modified_model.train_on_batch(x, y)
    skipgram_4words_modified_model_losses.append((ite, loss))
    print("skipgram 4words modified losses: {} -> {}".format(ite, loss))


skipgram 2words modified losses: 0 -> 40298.47370827198
skipgram 2words modified losses: 1 -> 36718.10098481178
skipgram 2words modified losses: 2 -> 35214.463769078255
skipgram 2words modified losses: 3 -> 33928.63531982899
skipgram 2words modified losses: 4 -> 32948.596044421196
skipgram 2words modified losses: 5 -> 32201.258150160313
skipgram 2words modified losses: 6 -> 31658.562999129295
skipgram 2words modified losses: 7 -> 31182.39734441042
skipgram 2words modified losses: 8 -> 30829.39433300495
skipgram 2words modified losses: 9 -> 30590.332844138145
skipgram 2words modified losses: 10 -> 30271.689700245857
skipgram 2words modified losses: 11 -> 29975.5885976851
skipgram 2words modified losses: 12 -> 29731.610060065985
skipgram 2words modified losses: 13 -> 29555.208632495254
skipgram 2words modified losses: 14 -> 29367.53787581995
skipgram 4words modified losses: 0 -> 40128.40839910507
skipgram 4words modified losses: 1 -> 37018.70784020424
skipgram 4words modified losses: 2 -

In [31]:
#Save computed vectors
VECTORS_SKIPGRAM_2WORDS_MODIFIED_FILENAME = 'vectors_skipgram_2words_modified.txt'
skipgram_2words_modified_model_vectors = skipgram_2words_modified_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_2WORDS_MODIFIED_FILENAME, skipgram_2words_modified_model_vectors)

VECTORS_SKIPGRAM_4WORDS_MODIFIED_FILENAME = 'vectors_skipgram_4words_modified.txt'
skipgram_4words_modified_model_vectors = skipgram_4words_modified_model.get_weights()[0]
save_computed_vectors(VECTORS_SKIPGRAM_4WORDS_MODIFIED_FILENAME, skipgram_4words_modified_model_vectors)

In [35]:
tokenizer.word_index

{'and': 2,
 'funny': 418,
 'hippopotamus': 906,
 'downward': 638,
 'peeped': 587,
 'hide': 1031,
 'changed': 235,
 'complained': 1076,
 'gave': 474,
 'histories': 705,
 'lap': 840,
 'conquest': 995,
 'family': 940,
 'softly': 960,
 'sad': 569,
 'his': 151,
 'pale': 961,
 'like': 47,
 'sharply': 471,
 'overhead': 664,
 'book': 205,
 'queer': 234,
 'gloves': 125,
 'yet': 929,
 'exactly': 1058,
 'mercia': 555,
 'cool': 692,
 'choked': 1079,
 'low': 186,
 'lodging': 897,
 'terrier': 947,
 'prize': 1060,
 'northumbria': 556,
 'crocodile': 845,
 'being': 362,
 'respectable': 765,
 'naturedly': 1153,
 'afterwards': 597,
 'time': 66,
 'paused': 1033,
 'who': 96,
 'purring': 933,
 'whose': 988,
 'opening': 779,
 'right': 158,
 'over': 100,
 'generally': 348,
 'considering': 382,
 'red': 715,
 'directions': 795,
 'myself': 786,
 'dear': 64,
 'sentence': 920,
 'or': 57,
 'day': 250,
 'rabbit': 63,
 'heard': 230,
 'confused': 1053,
 'next': 210,
 'chapter': 242,
 'shedding': 803,
 'carefully': 113

In [84]:
## train nearest neighbors model
nn_skipgram_2words = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_2words_model_vectors)
nn_skipgram_4words = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_4words_model_vectors)
nn_skipgram_2words_modified = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_2words_modified_model_vectors)
nn_skipgram_4words_modified = nn(n_neighbors=5, algorithm='brute', metric='cosine').fit(skipgram_4words_modified_model_vectors)


In [46]:
## --- SNIPPET USED TO UNDERSTAND SOME OPERATIONS ---- ##
print(skipgram_2words_modified_model_vectors.shape)
print(len(tokenizer.word_index))

(1183, 100)
1182


In [85]:
#Implement your own analogy function
# implementation of the function argmax(w.(w3 + w2 - w1)), with w - w3 =' w2 - w1
#from sklearn.metrics.pairwise import cosine_similarity
#from sklearn.neighbors import NearestNeighbors as nn
#nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(X)

def analogy(word1, word2, word3, word_vectors, nnmodel):
    vector_w1 = word_vectors[tokenizer.word_index[word1]]
    vector_w2 = word_vectors[tokenizer.word_index[word2]]
    vector_w3 = word_vectors[tokenizer.word_index[word3]]
    composed_vector = vector_w1 - vector_w2 + vector_w3
    distances, indices = nnmodel.kneighbors(composed_vector.reshape(1, -1))
    return (distances, indices)

Analyze the four different word embeddings

Implement your own function to perform the analogy task with.
    
Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    
Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.
    
Visualize your results and interpret your results


In [86]:
# I want to try with: melancholy - funny + smile = sad

(distances_skipgram_2words, indices_skipgram_2words) = analogy("cry", "funny", "smile", 
                                                               skipgram_2words_model_vectors, 
                                                              nn_skipgram_2words)

(distances_skipgram_4words, indices_skipgram_4words) = analogy("cry", "funny", "smile", 
                                                               skipgram_4words_model_vectors, 
                                                              nn_skipgram_4words)

In [88]:
#print(distances_skipgram_2words)
#print(indices_skipgram_2words)
for index in indices_skipgram_2words[0]:
    print(inverted_index[index])

smile
melancholy
game
soothing
patriotic


In [89]:
#print(distances_skipgram_4words)
#print(indices_skipgram_4words)
for index in indices_skipgram_4words[0]:
    print(inverted_index[index])

smile
melancholy
hide
yourself
circle


Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings